# JB2 - CIMR, RSIR and L1C

The goal of this notebook is to demonstrate how to perform an RSIR regridding on CIMR data to an EASE2 global grid. We will first outline the general steps for an L-band regrid. We will then compare this with a C and Ka band regrid. Finally, we will inspect the visual difference of the effect of changing the RSIR iteration parameter. 

> [**Note**]: Before running the code blocks below, the user should read and follow the steps outlined in the `README.md` file located inside the `notebooks` folder. Otherwise, the code may not run. 

# Step 1 - Import relevant modules

In [ ]:
import pathlib as pb

# Import Relevant Modules
from cimr_rgb.config_file import ConfigFile
from cimr_rgb.data_ingestion import DataIngestion
from cimr_rgb.regridder import ReGridder
from cimr_rgb.grid_generator import GRIDS
from cimr_rgb.product_generator import ProductGenerator

import copy
from numpy import full, nan, nanmean, isnan, isinf, polyfit

# You will need to configure matplotlib for your machine, you can probably remove the two lines below. 
import matplotlib # Remove this is plotting issues. 
tkagg = matplotlib.use('TkAgg') # Remove this if plotting issues. 
import matplotlib.pyplot as plt
plt.ion()

# Step 2 - Add your paths to the configuration file, load the configuration file and validate the configuration file 

In [ ]:
# Define path for configuration file of the first regrid.
config_1_path = pb.Path('./JB2.xml')

# Validate your config file
config_object = ConfigFile(config_1_path)

# You can inspect the parameters in the config file. There are additional parameters relevant to your particular configuration
# that are added as part of the validation. 
for attr, value in config_object.__dict__.items():
    print(f"{attr}: {value}")

# Step 2 - L-band

We will first outline the general steps for a regrid. When you run the code block below, you will notice that it will take a long time to process. This is an unfortunate fact of the RSIR algorithm. Run the cell below and have a look at the percentage progress at the bottom of the terminal. As you probably don't want to wait a few hours, we have functionality that allows you to only regrid a part of the grid for a quick inspection. 

After running the block below, if you don't want to wait for the entire granule to be processed, press the stop button at the top of the page and continue to the next block. (it may also be useful to comment this code block before continuing) 

In [ ]:
# We validated the config object in the block above, we will now use this config object to ingest data and perform a regrid

# Ingest Data 
# l1b_data = DataIngestion(config_object).ingest_data()

# Regrid data 
# l1c_l_band = ReGridder(config_object).regrid_data(l1b_data)

## `reduced_grid_inds`

We now want to only regrid a part of the output grid so we can do some quick inspection. In order to determine where on the EASE grid the data lands, and therefore select appropriate `reduced_grid_inds` parameters, we do a quick IDS regrid and inspect the result. 

> [**Note**]: The run can take some time; thus, if one wants a faster run, one should make the `reduced_grid_inds` smaller. 

In [ ]:
# We edit the config object to perform a IDS regrid. 

ids_config = copy.deepcopy(config_object)
ids_config.regridding_algorithm = 'IDS'


# Perform the IDS regrid 
l1b_data = DataIngestion(config_object).ingest_data()
l1c_ids = ReGridder(ids_config).regrid_data(l1b_data)

# Inspect the result on the EASE grid. 
variable = l1c_ids['L']['bt_h']
cell_row = l1c_ids['L']['cell_row']
cell_col = l1c_ids['L']['cell_col']

# Create grid
grid_shape = GRIDS[ids_config.grid_definition]['n_rows'], GRIDS[ids_config.grid_definition]['n_cols']
ease_grid = full(grid_shape, nan)
# Construct Grid 
for i in range(len(cell_row)):
    ease_grid[cell_row[i], cell_col[i]] = variable[i]

fig, axs = plt.subplots()
axs.imshow(ease_grid)
plt.show()


We can see from the image where the data lies on the grid and can now use the axes to select an appropriate smaller range to inspect data for the RSIR algorithm. We zoom into the region around around the border in the South of Mexico and choose rows from 540 to 600 and cols from 890 to 940. Now we need to update our config object and run again. 

In [ ]:
rsir_reduced_object = copy.deepcopy(ids_config)
rsir_reduced_object.regridding_algorithm = 'RSIR'
rsir_reduced_object.reduced_grid_inds = [560, 580, 900, 920]

l1c_rsir_reduced = ReGridder(rsir_reduced_object).regrid_data(l1b_data)

# Inspect the new data 
# Inspect the result on the EASE grid. 
variable = l1c_rsir_reduced['L']['bt_h']
cell_row = l1c_rsir_reduced['L']['cell_row']
cell_col = l1c_rsir_reduced['L']['cell_col']

# Create grid
grid_shape = GRIDS[ids_config.grid_definition]['n_rows'], GRIDS[ids_config.grid_definition]['n_cols']
ease_grid = full(grid_shape, nan)
# Construct Grid 
for i in range(len(cell_row)):
    ease_grid[cell_row[i], cell_col[i]] = variable[i]

fig, axs = plt.subplots()
axs.imshow(ease_grid[560:580, 900:920])
plt.show()

# Comparison of L band with C band and Ka band 

Note, the regridding of a few bands will take longer!

In [ ]:
multiband_config = copy.deepcopy(rsir_reduced_object)
multiband_config.target_band = ['L', 'C', 'KA']

l1b_multi = DataIngestion(multiband_config).ingest_data()
l1c_multi = ReGridder(multiband_config).regrid_data(l1b_multi)


In [ ]:
# plotting 

grid_shape = GRIDS[multiband_config.grid_definition]['n_rows'], GRIDS[multiband_config.grid_definition]['n_cols']
L_grid, C_grid, KA_grid = full(grid_shape, nan), full(grid_shape, nan), full(grid_shape, nan)

L_band_data = l1c_multi['L']
bt_h_l = L_band_data['bt_h']
cell_col_l = L_band_data['cell_col']
cell_row_l = L_band_data['cell_row']

for i in range(len(cell_col_l)):
    L_grid[cell_row_l[i], cell_col_l[i]] = bt_h_l[i]

C_band_data = l1c_multi['C']
bt_h_c = C_band_data['bt_h']
cell_col_c = C_band_data['cell_col']
cell_row_c = C_band_data['cell_row']

for i in range(len(cell_col_c)):
    C_grid[cell_row_c[i], cell_col_c[i]] = bt_h_c[i]

KA_band_data = l1c_multi['KA']
bt_h_ka = KA_band_data['bt_h']
cell_col_ka = KA_band_data['cell_col']
cell_row_ka = KA_band_data['cell_row']

for i in range(len(cell_col_ka)):
    KA_grid[cell_row_ka[i], cell_col_ka[i]] = bt_h_ka[i]

fig, axs = plt.subplots(1, 3)
im1 = axs[0].imshow(L_grid[560:580, 900:920], cmap = 'viridis')
axs[0].set_title('L_BAND')
im2 = axs[1].imshow(C_grid[560:580, 900:920], cmap = 'viridis')
axs[1].set_title('C_BAND')
im3 = axs[2].imshow(KA_grid[560:580, 900:920], cmap = 'viridis')
axs[2].set_title('KA_BAND')
colorbar = fig.colorbar(im1, ax=axs[0])
colorbar = fig.colorbar(im2, ax=axs[1])
colorbar = fig.colorbar(im3, ax=axs[2])
colorbar.set_label('BT [K]', fontsize=12)
plt.show()

# Inspect effect of RSIR Iteration. 

Now we will do three regrids of K band, but inspect a changing iteration parameter. 

In [ ]:
iteration_parameter = [1, 5, 15]

config_ka = copy.deepcopy(multiband_config)
config_ka.target_band = ['KA']

l1b_ka = DataIngestion(config_ka).ingest_data()

grid_shape = GRIDS[config_ka.grid_definition]['n_rows'], GRIDS[config_ka.grid_definition]['n_cols']


output_data = {}
for i in iteration_parameter:
    iteration_data = {}
    config = copy.deepcopy(config_ka)
    config.rsir_iteration = i
    l1c_ka = ReGridder(config).regrid_data(l1b_ka)
    bt_h = l1c_ka['KA']['bt_h']
    cell_row = l1c_ka['KA']['cell_row']
    cell_col = l1c_ka['KA']['cell_col']
    grid = full(grid_shape, nan)
    
    for sample in range(len(cell_row)):
        grid[cell_row[sample], cell_col[sample]] = bt_h[sample]

    output_data[str(i)] = grid

    

In [ ]:
# Plotting 
fig, axs = plt.subplots(1, 3)
im1 = axs[0].imshow(output_data['1'][560:580, 900:920], cmap = 'viridis')
axs[0].set_title('iteration = 1')
im2 = axs[1].imshow(output_data['5'][560:580, 900:920], cmap = 'viridis')
axs[1].set_title('iteration = 5')
im3 = axs[2].imshow(output_data['15'][560:580, 900:920], cmap = 'viridis')
axs[2].set_title('iteration = 15')
colorbar = fig.colorbar(im1, ax=axs[0])
colorbar = fig.colorbar(im2, ax=axs[1])
colorbar = fig.colorbar(im3, ax=axs[2])
colorbar.set_label('BT [K]', fontsize=12)
plt.show()